In [1]:
# Fasttext main branch has issue in calculation of F1 score. hence using this label from github
!pip install git+https://github.com/facebookresearch/fastText.git@b64e359d5485dda4b4b5074494155d18e25c8d13 -q

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import fasttext

In [3]:
def write_ft_file(X,y,fname):
    with open(fname, 'w',encoding="utf-8") as f:
        for i,r in X.iteritems():
            f.write(f'__label__{y[i]} {X[i]}')
            f.write('\n')

In [6]:
# Load MP3 mapped data
data = pd.read_json('/content/drive/MyDrive/song_lyric_map.json')

In [8]:
counts = data['Ragam'].value_counts()
low_vals = counts[counts<10]
for i, l in enumerate(low_vals):
  data = data[data['Ragam'] != low_vals.index[i]]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data['Lyric'], data['Ragam'], random_state=42, test_size=0.3)

In [10]:
write_ft_file(X_train, y_train, 'traindata.txt')
write_ft_file(X_test, y_test, 'testdata.txt')

In [11]:
# Auto tune Fast Text Model
ft_auto_tune_model = fasttext.train_supervised(input="traindata.txt", 
                                    autotuneValidationFile='testdata.txt', 
                                    autotuneModelSize="1M", 
                                    autotuneMetric='f1',
                                    dim=50)

In [12]:
test_score = ft_auto_tune_model.test('testdata.txt')
train_score = ft_auto_tune_model.test('traindata.txt')
print('Training')
print(train_score)
print('Validation')
print(test_score)

Training
(798, 0.7944862155388471, 0.7944862155388471)
Validation
(342, 0.10818713450292397, 0.10818713450292397)


In [13]:
ft_auto_tune_model.save_model('ft_autotuned_model_song_lyrics_raga.bin')